<a href="https://colab.research.google.com/github/sokrypton/ColabDesign/blob/v1.1.1/af/examples/af_pseudo_diffusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#AF_pseudo_diffusion
Hacking AlphaFold to be a diffusion model (for backbone generation)

In [ ]:
#@title setup
%%time
import os
if not os.path.isdir("params"):
  # get code
  os.system("pip -q install git+https://github.com/sokrypton/ColabDesign.git@v1.1.1")
  # for debugging
  os.system("ln -s /usr/local/lib/python3.7/dist-packages/colabdesign colabdesign")
  # download params
  os.system("mkdir params")
  os.system("apt-get install aria2 -qq")
  os.system("aria2c -q -x 16 https://storage.googleapis.com/alphafold/alphafold_params_2022-03-02.tar")
  os.system("tar -xf alphafold_params_2022-03-02.tar -C params")

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import os
from colabdesign import mk_afdesign_model, clear_mem
from IPython.display import HTML
from google.colab import files
import numpy as np

CPU times: user 1.3 s, sys: 371 ms, total: 1.67 s
Wall time: 2min 22s


In [ ]:
clear_mem()
af_model = mk_afdesign_model(protocol="hallucination", use_templates=True)
af_model.prep_inputs(length=256)

print("length",af_model._len)
print("weights",af_model.opt["weights"])

length 256
weights {'con': 1.0, 'exp_res': 0.0, 'helix': 0.0, 'pae': 0.0, 'plddt': 0.0, 'seq_ent': 0.0}


In [ ]:
af_model.restart()
af_model._inputs["batch"] = {"aatype": af_model._params["seq"].argmax(-1)[0],
                             "all_atom_mask":np.tile(np.arange(37)[None] < 4, (af_model._len,1)).astype(float),
                             "all_atom_positions":np.tile(5.5 * np.arange(af_model._len)[:,None,None], (1,37,3))}
for k in range(100):
  
  # add noise
  w = 1 - k/100
  random_noise = np.random.normal(size=(af_model._len,37,3)) * w * 1
  af_model._inputs["batch"]["all_atom_positions"] += random_noise

  # denoise
  aux = af_model.predict(return_aux=True, verbose=False)
  af_model._inputs["batch"]["all_atom_positions"] = af_model.aux["atom_positions"]
  af_model._save_results(aux)
  af_model._k += 1

1 models [0] recycles 0 hard 1 soft 0 temp 1 loss 3.92 con 3.92 plddt 0.29 ptm 0.19
2 models [0] recycles 0 hard 1 soft 0 temp 1 loss 3.78 con 3.78 plddt 0.31 ptm 0.25
3 models [0] recycles 0 hard 1 soft 0 temp 1 loss 3.62 con 3.62 plddt 0.36 ptm 0.30
4 models [0] recycles 0 hard 1 soft 0 temp 1 loss 3.48 con 3.48 plddt 0.38 ptm 0.34
5 models [0] recycles 0 hard 1 soft 0 temp 1 loss 3.37 con 3.37 plddt 0.42 ptm 0.38
6 models [0] recycles 0 hard 1 soft 0 temp 1 loss 3.25 con 3.25 plddt 0.46 ptm 0.43
7 models [0] recycles 0 hard 1 soft 0 temp 1 loss 3.28 con 3.28 plddt 0.44 ptm 0.41
8 models [0] recycles 0 hard 1 soft 0 temp 1 loss 3.25 con 3.25 plddt 0.45 ptm 0.42
9 models [0] recycles 0 hard 1 soft 0 temp 1 loss 3.18 con 3.18 plddt 0.46 ptm 0.44
10 models [0] recycles 0 hard 1 soft 0 temp 1 loss 2.91 con 2.91 plddt 0.54 ptm 0.49
11 models [0] recycles 0 hard 1 soft 0 temp 1 loss 2.92 con 2.92 plddt 0.53 ptm 0.48
12 models [0] recycles 0 hard 1 soft 0 temp 1 loss 2.85 con 2.85 plddt 0.5

In [ ]:
af_model.plot_pdb()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [108]:
HTML(af_model.animate(dpi=100))